## Testing CSP Algorithm
This notebook is used to create a reviewed flight schedule. It should take the input:
* the original departure / landing time\
* id of the flight

In [81]:
import pandas as pd
import constraint
import time

In [6]:
data = pd.read_csv("./data/19DEC2023_AMS.csv", parse_dates= ["time_sch","time_act"])

<ipython-input-6-9820427f3e9e>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv("./data/19DEC2023_AMS.csv", parse_dates= ["time_sch","time_act"])
<ipython-input-6-9820427f3e9e>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv("./data/19DEC2023_AMS.csv", parse_dates= ["time_sch","time_act"])


Note that the data contains flights scheduled for the morning hours to the earning hours of the preceding day.
We will first sort the values by the scheduled time `time_sch` to identify those late night flights

In [11]:
data_sorted = data.sort_values("time_sch").reset_index(drop = True)

For the purpose of testing, we will only subset a small chunk of the flight schedule.

In [111]:
# retrieve the boolean index for flightrs scheduled between 10pm - 3am
# index = data_sorted['time_sch'].map(lambda x: x.hour>=  23) 
# subset the data
# data_subset = data[index]
# print(f"There are {len(data_subset)} scheduled flights in this dataset.")


# alternatively get the tail
data_subset_n8 = data_sorted.tail(8)


**Assumptions**
Now assume that the airport has to be shut down for 30 minutes between 22:00 to 22:30. Note also that after 23:00, the airport can only operate on 1 runway. We also assume the airport only have enough staff to handle any take-off and landing until 01:00 for simplicity.
We also assume that flights can at best arrive on schedule but never earlier than schedule.
We will **not** consider if airborne flight has enough fuel to cruise in the air or not for this first simulation.

We will first run a naive algorithm that test for the `constraint` package in solving the question. 
Each flight can be considered as a varaible. For simplicity, we express the domain as the time (in minute) after the airport continue to operate (22:30). Given the assumption we made, the domain should take the minimum value of $f(x)=max(0,time\_sch)$ and the maximum value of 150

In [113]:
# initiate the problem
flight_schedule = constraint.Problem()

# add a variable for each flight
for key, flight in data_subset_n8.iterrows():
    print(f"Adding variable for {flight['code']} that was scheduled to use the runway at {flight['time_sch'].hour}:{flight['time_sch'].minute:02d}")
    # compute the relative time of schedule take-off/landing
    min_time = max((flight['time_sch'].hour - 23) * 60 + flight['time_sch'].minute -30,0)
    flight_schedule.addVariable(flight['code'], range(min_time,30,5))

Adding variable for KL 1082 KLM that was scheduled to use the runway at 23:20
Adding variable for KL 1136 KLM that was scheduled to use the runway at 23:20
Adding variable for KL 1834 KLM that was scheduled to use the runway at 23:20
Adding variable for HV 6118 Transavia that was scheduled to use the runway at 23:25
Adding variable for KL 1608 KLM that was scheduled to use the runway at 23:25
Adding variable for HV 6902 Transavia that was scheduled to use the runway at 23:40
Adding variable for HV 5356 Transavia that was scheduled to use the runway at 23:45
Adding variable for HV 6120 Transavia that was scheduled to use the runway at 23:50


In [114]:
# define constraint - no more than 2 flight at a given time 
def not_same_time(*args):
    """
    check that no two flights are using the runway at the same time
    """
    schedule = sorted([agr for agr in args])
    schedule_counter = pd.Series(schedule).value_counts()
    if schedule_counter.max() >=2:
        return None
    else:
        return True


In [115]:
flight_schedule.addConstraint(not_same_time,[flight['code'] for key, flight in  data_subset_n8.iterrows()])

In the case of only 5 flights, the algorithm returns 48 solutions in about 10 secods. For   As the number of flight increases, the processing time to get all the solution increases exponentially.